In [11]:
from pyspark import SparkContext
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_6153/414631668.py:2 

In [ ]:
!head -1 201704_Usage_Bicimad.json

In [12]:
import json
import datetime

In [13]:
def mapper(line):
    data = json.loads(line)
    usert = data['user_type']
    time = data['travel_time']
    date = convert_date(data['unplug_hourTime'])
    return usert, time, date

In [14]:
def convert_date(date):
    if isinstance(date, str):
        date = date.replace("Z","+0000")
        date = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
    else:
        date = date['$date']
        date = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.%f%z")
    return (date.month,date.hour)

In [15]:
convert_date({ "$date" : "2019-06-01T12:40:00.000+0200" })

(6, 12)

In [16]:
rdd_base = sc.textFile('sample01.json')

In [17]:
rdd = rdd_base.map(mapper)

In [18]:
rdd.countByKey()
#0: no ha sido posible determinarlo, 1: pase anual, 2: ocasional, 3: empresa

defaultdict(int, {1: 1707, 3: 130, 0: 206, 2: 7})

In [20]:
rdd_user1 = rdd.filter(lambda x: x[0]==1).map(lambda x: (x[2][0],x[1],x[2][1]))#mes, tiempo(s), hora(24)
n1 = rdd_user1.count()
print(n1)
rdd_user2 = rdd.filter(lambda x: x[0]==2).map(lambda x: (x[2][0],x[2][1],x[1]))#mes, hora 24h, tiempo seg.
n2 = rdd_user2.count()
print(n2)
rdd_user3 = rdd.filter(lambda x: x[0]==3).map(lambda x: (x[2][0],x[1],x[2][1]))
n3 = rdd_user3.count()
print(n3)

1707
7
130


In [21]:
rdd_user2.take(7)

[(10, 17, 2409),
 (10, 10, 392),
 (10, 11, 9236),
 (10, 12, 890),
 (10, 11, 735),
 (10, 17, 2036),
 (10, 10, 3239)]